In [1]:
import time
import glob
import pandas as pd

import requests

import pytz
import numpy as np
import networkx as nx

import pylab
from colour import Color
from pylab import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import bigbang
from bigbang.ingress.listserv import ListservMailList, ListservMessageParser
from bigbang.analysis.listserv import ListservMailList as AnaListservMailList

from bigbang.ingress.w3c import W3CMailList, W3CMessageParser
from bigbang.ingress.utils import (
    get_website_content,
    set_website_preference_for_header,
    get_auth_session,
)

plt.style.use("publication_jcap.mplstyle")

from bs4 import BeautifulSoup

In [2]:
mlist = AnaListservMailList.from_mbox(
    name="3GPP_TSG_SA_WG3_LI",
    #filepath="/home/christovis/InternetGov/proj1_3gpp_and_comp/bin/test.mbox",
    filepath="/home/christovis/InternetGov/bigbang/archives/3GPP/3GPP_TSG_SA_WG3_LI.mbox",
)

In [19]:
col_attachment = [col for col in mlist.df.columns if col.startswith('attachment-')]
for msg_idx, msg in mlist.df.iterrows():
    if len(msg[col_attachment].dropna()) > 0:
        print
(' ').join(list(msg[col_attachment].dropna().values))
        break

3GPP TSG-SA3 Meeting #84-LI-e-a s3i220038r5 Online, , 24th Jan 2022 - 28th Jan 2022      First change   7.11.2.2 Signature generation The IRI-POI present in the Telephony AS or IBCF, shall generate an xIRI containing a STIRSHAKENSignatureGeneration record when the following conditions are met: - Telephony AS or IBCF is interacting with the SIGNING AS. Whether it is the Telephony AS or IBCF for sessions is based on network configuration and local policy of the CSP. Whether it is IBCF for MSISDN-less SMS is based on network configuration and local policy of the CSP. - P-Asserted Identity or From header of SIP INVITE or SIP MESSAGE request received from S-CSCF is a target identity. - A PASSporT is received from the SIGNING AS and is included in an outgoing SIP INVITE or SIP MESSAGE request in a SIP Identity header. - Based on RFC 8946 [76], SIP retargeting creates a new Identity header signed by the SIGNING AS and containing the "div" claim to attach to the session.  The following table c

In [18]:
msg[col_attachment].dropna().values

array(['3GPP TSG-SA3 Meeting #84-LI-e-a s3i220038r5 Online, , 24th Jan 2022 - 28th Jan 2022      First change   7.11.2.2 Signature generation The IRI-POI present in the Telephony AS or IBCF, shall generate an xIRI containing a STIRSHAKENSignatureGeneration record when the following conditions are met: - Telephony AS or IBCF is interacting with the SIGNING AS. Whether it is the Telephony AS or IBCF for sessions is based on network configuration and local policy of the CSP. Whether it is IBCF for MSISDN-less SMS is based on network configuration and local policy of the CSP. - P-Asserted Identity or From header of SIP INVITE or SIP MESSAGE request received from S-CSCF is a target identity. - A PASSporT is received from the SIGNING AS and is included in an outgoing SIP INVITE or SIP MESSAGE request in a SIP Identity header. - Based on RFC 8946 [76], SIP retargeting creates a new Identity header signed by the SIGNING AS and containing the "div" claim to attach to the session.  The following

In [6]:
col_attachment

['attachment-0',
 'attachment-1',
 'attachment-2',
 'attachment-3',
 'attachment-4']

In [20]:
for msg_idx, msg in mlist.df.iterrows():
    print(msg.columns)
    break

'hello\nmy\t name is\r adam\x07.'

In [18]:
from urllib.parse import urljoin, urlparse
from urllib.request import urlretrieve

list_name = "3GPP_TSG_SA_WG3_LI"
url = "https://list.etsi.org/scripts/wa.exe?A2=3GPP_TSG_SA_WG3_LI;f1d4996.2102D&S="
#url = "https://list.etsi.org/scripts/wa.exe?A2=3GPP_TSG_SA_WG3_LI;6ef9e74e.1702D&S="
url_root = ("/").join(url.split("/")[:-2])
url_login = "https://list.etsi.org/scripts/wa.exe?LOGON"
login = {"username": "nielsto@gmail.com", "password": "BigBang11!"}

session = get_auth_session(url_login, **login)
soup = get_website_content(url, session=session)

=========> success <=========


In [20]:
a_tags = soup.select(f'a[href*="A3="][href*="{list_name}"]')
href_plain_text = [
    tag.get("href") for tag in a_tags if "Fplain" in tag.get("href")
]
href_html_text = [
    tag.get("href") for tag in a_tags if "Fhtml" in tag.get("href")
]
href_html_attachment = [
    tag.get("href") for tag in a_tags if "Fvnd" in tag.get("href")
]

In [21]:
print(urljoin(url_root, href_html_text[0]))
print()
print(urljoin(url_root, href_html_attachment[0]))

https://list.etsi.org/scripts/wa.exe?A3=ind2102D&L=3GPP_TSG_SA_WG3_LI&E=quoted-printable&P=1314&B=--_000_LNXP123MB24602C6F1B65AC6C35EB644B949D9LNXP123MB2460GBRP_&T=text%2Fhtml;%20charset=iso-8859-1&XSS=3&X=O89BB392EE92150D7C9&Y=nielsto%40gmail.com&header=1

https://list.etsi.org/scripts/wa.exe?A3=ind2102D&L=3GPP_TSG_SA_WG3_LI&E=base64&P=3522&B=--_004_LNXP123MB24602C6F1B65AC6C35EB644B949D9LNXP123MB2460GBRP_&T=application%2Fvnd.openxmlformats-officedocument.wordprocessingml.document;%20name=%22draft s3i210102 Agenda.docx%22&N=draft s3i210102 Agenda.docx&attachment=q&XSS=3&X=O89BB392EE92150D7C9&Y=nielsto%40gmail.com


In [32]:
from urllib.parse import urljoin, urlparse
from urllib.request import urlretrieve
from docx import Document
from io import BytesIO
import requests
import re

url_attachment = urljoin(url_root, href_html_attachment[0])
docx = Document(BytesIO(requests.get(url_attachment).content))
document = ('\n').join([para.text for para in docx.paragraphs])
filename = re.search("\%22(.*?)\%22", url_attachment).group(1)
subtype = filename.split('.')[-1]

In [33]:
from collections import namedtuple

doc = namedtuple('attachment', 'text subtype filename')
attachment = doc(text=document, subtype=subtype, filename=filename)

In [2]:
msg_parser = ListservMessageParser(
    website=True,
    url_login="https://list.etsi.org/scripts/wa.exe?LOGON",
    url_pref="https://list.etsi.org/scripts/wa.exe?PREF",
    login={"username": "nielsto@gmail.com", "password": "BigBang11!"},
)
msg = msg_parser.from_url(
    list_name="3GPP_TSG_SA_WG3_LI",
    url="https://list.etsi.org/scripts/wa.exe?A2=3GPP_TSG_SA_WG3_LI;f1d4996.2102D&S=",
    fields="total",
)
msg.get_payload()

=========> success <=========
href_html_attachment :  1


'Hi All,\n\nPlease find attached the draft agenda for next week. As a reminder, please ma=\nke sure that you have registered on 3GU before contributing to the meeting.\n\nRegards,\nAlex.\n\n\n'

In [3]:
msg['message-id']

'<LNXP123MB24602C6F1B65AC6C35EB644B949D9@LNXP123MB2460.GBRP123.PROD.OUTLOOK.COM>'

In [4]:
msg['subject']

"'SA3#80-LI-e-b Draft Agenda'"

In [5]:
msg['Content-Disposition']

'3GPP TSG-SA3LI Meeting #80e-b draft S3i210102 E-meeting 2nd March – 4th March 2021   Source: Chairman of 3GPP TSG SA WG3 LI Sub WG Title: Agenda for SA3#80-LI-e-b Document for: Approval Agenda Item: 2  1 Welcome Meeting Start on the SA3-LI email reflector 2nd March 2020 14:00 ETSI Time.  At meeting start the Agenda and previous meeting report will automatically be approved and all documents in agenda items 2 and 3 noted without further review on the email reflector.  Deadlines as of Meeting Start:  Start of e-meeting:  2nd March   14:00 ETSI Time Official Conf Calls  3rd / 4th March   14:00 – 16:30 ETSI Time Email List Last comment: 4th March   16:00 ETSI Time End of e-meeting:  4th March   17:00 ETSI Time  Conf Call link:   Conference Call Agenda / Initial Timing Allocations   Please note that actual timing and allocation of individual agenda items is subject to change and update during the e-meeting, in order to best utilise the time available.  Based on the input to the meeting it 

In [2]:
msg_parser = W3CMessageParser(
    website=True,
)
msg = msg_parser.from_url(
    list_name="public-2018-permissions-ws",
    url="https://lists.w3.org/Archives/Public/public-2018-permissions-ws/2019May/0000.html",
    fields="total",
)

In [3]:
mlist = W3CList.from_url(
    name="public-bigdata",
    url="https://lists.w3.org/Archives/Public/public-bigdata/",
    select={
        "years": 2015,
        "months": "August",
        "fields": "header",
    },
)
len(mlist)

public-bigdata: 100%|##########| 3/3 [00:06<00:00,  2.12s/it]


3

In [10]:
mlist.to_pandas_dataframe()

,message-id,date,to,cc,subject,from,archived-at,body
0,<55D47CD4.8040809@cs.helsinki.fi>,2015-08-19 15:55:48+03:00,public-bigdata@w3.org,,Advances in High Dimensional Big Data 2015 CFP...,Sotirios Tasoulis <tasoulis@cs.helsinki.fi>,<https://lists.w3.org/Archives/Public/public-b...,None
1,<55C85D12.2020207@cs.helsinki.fi>,2015-08-10 11:13:06+03:00,public-bigdata@w3.org,,Advances in High Dimensional Big Data 2015 CFP...,Sotirios Tasoulis <tasoulis@cs.helsinki.fi>,<https://lists.w3.org/Archives/Public/public-b...,None
2,<55B9EF83.1020802@cs.helsinki.fi>,2015-07-30 12:33:55+03:00,public-bigdata@w3.org,,Advances in High Dimensional Big Data 2015 CFP...,Sotiris Tasoulis <tasoulis@cs.helsinki.fi>,<https://lists.w3.org/Archives/Public/public-b...,None


In [4]:
url = "https://lists.w3.org/Archives/Public/public-bigdata/"
soup = bigbang.w3c.get_website_content(url)

periods = []
urls_of_periods = []
rows = soup.select("tbody tr")
for row in rows:
    link = row.select("td:nth-of-type(1) a")[0]
    periods.append(link.text)
    urls_of_periods.append(url + link.get("href"))


print(periods[:3])
print(urls_of_periods[:3])

['September 2021', 'August 2021', 'January 2021']
['https://lists.w3.org/Archives/Public/public-bigdata/2021Sep/', 'https://lists.w3.org/Archives/Public/public-bigdata/2021Aug/', 'https://lists.w3.org/Archives/Public/public-bigdata/2021Jan/']


In [5]:
df[["Country"]].to_csv("/home/christovis/InternetGov/test.csv", index=False)

NameError: name 'df' is not defined

In [ ]:
df[["Organization long name"]].to_csv("/home/christovis/InternetGov/test.csv", index=False)

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_website_content(
    url: str,
):
    sauce = requests.get(url)
    assert sauce.status_code == 200
    soup = BeautifulSoup(sauce.content, "html.parser")
    return soup

soup = get_website_content("https://list.etsi.org/scripts/wa.exe?A0=3GPP_TSG_CT_WG6")

In [ ]:
ListservList.get_messages_urls(
    name="3GPP_TSG_GERAN_WG4_PCS1900",
    url="https://list.etsi.org/scripts/wa.exe?A1=ind1208&L=3GPP_TSG_GERAN_WG4_PCS1900",
)

In [ ]:
ListservList.get_all_periods_and_their_urls(
    url="https://list.etsi.org/scripts/wa.exe?A0=3GPP-REVIEW",
)

In [ ]:
if "No Recent Messages Available" in str(requests.get("https://list.etsi.org/scripts/wa.exe?A0=3GPP_TSG_CT_WG6").content):
    print('yes')

In [ ]:
if "Latest Messages" in str(requests.get("https://list.etsi.org/scripts/wa.exe?A0=3GPP_TSG_CT_WG6").content):
    print('yes')

In [ ]:
for li in str(requests.get("https://list.etsi.org/scripts/wa.exe?A0=3GPP_TSG_CT_WG6").content):
    if "Latest Messages" in li:
        print(li)


In [ ]:
#selec = soup.find_all('table', attrs={'width':"100%", 'class':'tableframe', 'cellspacing':"0", 'cellpadding':"5", 'border':"0"})
selec = soup.find_all('span', attrs={'id':"output"}) #, 'style':"visibility: visible;"})
selec

In [ ]:
selec[0].descendants

In [ ]:
selec = soup.find_all('table', attrs={'width':"100%", 'class':'tableframe', 'cellspacing':"0", 'cellpadding':"5", 'border':"0"})
selec.get_text()
#if "No Recent Messages Available" in str(selec):
#    print("yes")

In [ ]:
len(march.lists)

In [ ]:
textfile = open("/home/christovis/AGE/datactive/bigbang/examples/url_collections/listserv.3GPP.txt", "w")
for element in march.lists:
    textfile. write(element)
textfile. close()

In [ ]:
res = ListservList.get_message_urls(
    name="3GPP_TSG_GERAN_WG4_PCS1900",
    url="https://list.etsi.org/scripts/wa.exe?A0=3GPP_TSG_GERAN_WG4_PCS1900",
    select={"fields": "header"},
)
len(res)

In [ ]:
dir_data = "/home/christovis/AGE/datactive/bigbang-archives/3GPP/"
files_scraped = glob.glob(dir_data + "*.mbox")
files_scraped = [url.split('/')[-1].split('.')[0] for url in files_scraped]

files_avail = []
for url in open("/home/christovis/AGE/datactive/bigbang/examples/url_collections/listserv.3GPP.txt"):
    if url.startswith("#"):
        continue
    elif len(url) == 0:
        continue
    files_avail.append(url)

files_missing = [url for url in files_avail if url.strip().split('A0=')[-1] not in files_scraped]

textfile = open("/home/christovis/AGE/datactive/listserv_missing.3GPP.txt", "w")
for element in files_missing:
    textfile. write(element)
textfile. close()

In [ ]:
length = []
for url in files_missing[:]:
    name = url.split("A0=")[-1].strip()
    print(name)
    url = url.strip()
    length.append(len(
        ListservList.get_message_urls(
            name=name,
            url=url,
            select={"fields": "header"},
        )
    ))

In [ ]:
textfile = open("/home/christovis/AGE/datactive/listserv_missing.3GPP.txt", "w")
for element in files_missing:
    textfile. write(element)
textfile. close()

In [ ]:
color = [
    Color(rgb=(np.array([5, 102, 141])/255)),
    Color(rgb=(np.array([67, 170, 139])/255)),
    Color(rgb=(np.array([248, 150, 30])/255)),
    Color(rgb=(np.array([249, 65, 68])/255)),
    Color(rgb=(np.array([155, 89, 182])/255)),
    Color(rgb=(np.array([1, 1, 1])/255)),
]
color = mpl.colors.ListedColormap([c.rgb for c in color])

In [ ]:
msg_parser = ListservMessageParser(
    website=True,
    login={"username": "nielsto@gmail.com", "password": "BigBang11!"},
)
msg = msg_parser.from_url(
    list_name="3GPP_TSG_RAN_DRAFTS",
    url="https://list.etsi.org/scripts/wa.exe?A2=ind2010B&L=3GPP_TSG_RAN_DRAFTS&O=D&P=29883",
    fields="total",
)

In [ ]:
mlist = ListservList.from_mbox(
    name="AIOTIWG8",
    filepath="/home/christovis/AGE/datactive/bigbang-archives/3GPP/AIOTIWG8.mbox",
)
df_mlist = mlist.to_pandas_dataframe()
df_mlist["date"] = pd.to_datetime(df_mlist["date"], format="%a, %d %b %Y %H:%M:%S %z")

In [ ]:
df_mlist['date']

In [ ]:
pd.to_datetime(df_mlist["date"], format="%a, %d %b %Y %H:%M:%S %z").max()

In [ ]:
import datetime

datetime.datetime.strptime(
    df_mlist['date'].values[11], "%a, %d %b %Y %H:%M:%S %z"
)

In [ ]:
detector = gd.GenderDetector('us')

dic_ng = {"name": [], "gender": []}
genders = []
for name_addr in df_arch["from"].values[:40]:
    name, addr = email.utils.parseaddr(name_addr)
    dic_ng["name"].append(name)
    try:
        forename = name.split(" ")[0]
        dic_ng["gender"].append(detector.guess(forename))
    except:
        dic_ng["gender"].append('others')

df_ng = pd.DataFrame.from_dict(dic_ng)
df_ng.groupby('gender').count()

In [ ]:
trans = {"female": "Woman", "male": "Man", "unknown": "Others"}

gender = df_ng.groupby('gender').count().index.values
count = df_ng.groupby('gender').count()["name"].values
count = count / np.sum(count)
count

gender = [trans[g] for g in gender]

In [ ]:
fig, ax = plt.subplots(
    1, 1,
    figsize=(4, 4),
)

nr_colors = len(gender) + 3
cmap = cm.binary(np.arange(nr_colors) / nr_colors)

ax.pie(
    count,
    radius=3.,
    labels=gender,
    labeldistance=1.2,
    autopct='%1.f%%',
    shadow=False,
    startangle=90,
    colors=cmap,
)
ax.axis('equal')

plt.savefig(
    "IEEE_gender.png",
    format='png',
    transparent=True,
    dpi=300,
    bbox_inches='tight',
)
plt.show()

In [ ]:
series = []  
for (url, ml) in mlists:
    activity = Archive.get_activity(Archive(ml)).sum(0)
    activityFrame = pd.DataFrame(activity, columns=['Message Count'])
    
    activityFrame['Name'] = activityFrame.index.map(lambda x: parse.clean_from(x))    
    activityFrame['First Name'] = activityFrame['Name'].map(lambda x: parse.guess_first_name(x))
    activityFrame['Guessed Gender'] = activityFrame['First Name'].map(guess_gender)
    
    activityFrame.to_csv(('senders_guessed_gender-%s.csv' % ml_shortname(url)),encoding='utf-8')
    
    counts = activityFrame.groupby('Guessed Gender')['Message Count'].count()
    counts.name=url
    series.append(counts)

pd.DataFrame(series)

In [ ]:
march = ListservArchive.from_mbox(
    name="IEEE",
    directorypath="/home/christovis/AGE/datactive/bigbang-archives/IEEE/",
)
df_arch = march.to_pandas_dataframe()
df_arch = df_arch.set_index('date')
df_arch.index = pd.to_datetime(df_arch.index)
#df_mlist["data"] = df_mlist["data"].astype('datetime64[ns]')

In [ ]:
df_dt = df_mlist.groupby(df_mlist.index.to_period("M")).count()

fig, ax = plt.subplots(
    figsize=(24, 4),
)

plt.bar(
    np.arange(len(df_dt.index.values)),
    height=df_dt["From"].values,
    color=color(0),
)

ax.set_xlim(0, len(df_dt.index.values))
xlabels = ax.get_xticks().tolist()
xlabels = list(map(int, xlabels))
xlabels = df_dt.index.values[xlabels[:-1]]
ax.set_xticklabels(xlabels, rotation=90)
ax.set_ylabel('Nr. of Messages')

# Network

In [ ]:
IG = nx.DiGraph()

from_dict = {}
sender_counts = {}
reply_counts = {}

for index, msg in df.iterrows():
    name, addr = email.utils.parseaddr(msg["from"])
    
    from_dict[index] = name
    sender_counts[name] = sender_counts.get(name, 0) + 1
    reply_counts[name] = {}
    IG.add_node(name)
    
for sender, count in list(sender_counts.items()):
    IG.nodes[sender]["sent"] = count
    
replies = [msg for index, msg in df.iterrows() if msg["reply-to"] is not None]

In [ ]:
for msg in replies:
    m_from = msg["from"]

    reply_to_mid = msg["reply-to"]

    if reply_to_mid in from_dict:
        m_to = from_dict[reply_to_mid]
        reply_counts[m_from][m_to] = reply_counts[m_from].get(m_to, 0) + 1
        
for m_from, edges in list(reply_counts.items()):
    for m_to, count in list(edges.items()):
        print(count)
        IG.add_edge(m_from, m_to, weight=count)

In [ ]:
options = {
    'edge_color': color(3),
    'node_color': color(1),
    'node_size': 50,
    'width': 1,
}

nx.draw(
    IG,
    with_labels=False,
    **options,
)

mlistmlist = ListservList.from_mbox(
    name="3GPP_TSG_RAN_WG5_IOT",
    filepath="/home/christovis/AGE/datactive/bigbang-archives/3GPP/3GPP_TSG_RAN_WG5_IOT.mbox",
)
df_mlist = mlist.to_pandas_dataframe()

affiliations = []
for name_addr in df_mlist["from"].values:
    name, addr = email.utils.parseaddr(name_addr)
    aff  = addr.split('@')[-1]
    affiliations.append(aff)
    
name = []
count = []
for aff in list(set(affiliations)):
    name.append(aff)
    count.append(affiliations.count(aff))
    
count = np.array(count) / np.sum(count)# Affiliations

In [ ]:
mlist.messages[0].keys()

In [ ]:
mlist = ListservList.from_mbox(
    name="3GPP_TSG_SA_WG4",
    filepath="/home/christovis/AGE/datactive/bigbang-archives/3GPP/3GPP_TSG_SA_WG4.mbox",
)

df_mlist = mlist.to_pandas_dataframe(include_body=False)

affiliations = []
for name_addr in df_mlist["from"].values:
    name, addr = email.utils.parseaddr(name_addr)
    aff  = addr.split('@')[-1]
    affiliations.append(aff)
    
name = []
count = []
for aff in list(set(affiliations)):
    name.append(aff)
    count.append(affiliations.count(aff))
    
#count = np.array(count) / np.sum(count)

indx = np.argsort(count).astype(int)
name = [name[ii] for ii in indx]
count = np.sort(count)

In [ ]:
for n, c in zip(name, count):
    print(f"{n} {c}")

In [ ]:
mlist = ListservList.from_mbox(
    name="3GPP_TSG_SA_WG4",
    filepath="/home/christovis/AGE/datactive/bigbang-archives/3GPP/3GPP_TSG_SA_WG4.mbox",
)
df_mlist = mlist.to_pandas_dataframe(include_body=False)

dic_affcon = get_contributions_per_affiliations(df_mlist, percentage=True, contract=0.03)

In [ ]:
dic_affcon

In [ ]:
fig, ax = plt.subplots(
    1, 1,
    figsize=(6, 6),
)

nr_colors = len(dic_affcon.values()) + 10
cmap = cm.binary(np.arange(nr_colors) / nr_colors)

ax.pie(
    dic_affcon.values(),
    radius=3.,
    labels=dic_affcon.keys(),
    labeldistance=1.2,
    autopct='%1.1f%%',
    shadow=False,
    startangle=90,
    colors=cmap,
)
ax.axis('equal')


"""
plt.savefig(
    "3GPP_TSG_RAN_WG5_IOT_affiliations.png",
    format='png',
    transparent=True,
    dpi=300,
    bbox_inches='tight',
)
plt.show()"""

In [ ]:
a = np.ones((2,2))
b = np.ones((2,2)) * 2

In [ ]:
np.linalg.inv(np.array([[1., 2.], [3., 4.]]))

In [ ]:
b